In [25]:
import os 
import re
import datetime

import pandas as pd
import numpy as np

from typing import List, Tuple, Union
from sqlalchemy import create_engine, text
 
from tqdm.auto import tqdm
tqdm.pandas()

In [26]:
from dotenv import load_dotenv
load_dotenv()

True

In [27]:
username = os.environ["USERNAME"]
password = os.environ["PASSWORD"]
hostname = os.environ["HOSTNAME"]
database = "ctrial"
port = os.environ["PORT"]

In [28]:
df = pd.read_csv("ctgov_20230608_instruct_embedding.csv")

In [29]:
df

,nct_id,brief_title,official_title,baseline_measurements,brief_summaries,detailed_descriptions,criteria,gender,minimum_age,maximum_age,facilities,city,state,zip,country,recruitment_details,pre_assignment_details,study_type,embeddings
0,NCT00000143,Studies of Ocular Complications of AIDS (SOCA)...,Studies of Ocular Complications of AIDS (SOCA)...,NaN,"To compare the newest CMV retinitis drug, cido...",Cytomegalovirus (CMV) is among the most freque...,Inclusion criteria:\n\nAge 13 years or older\n...,All,13 Years,NaN,"University of South Florida, MDC Box 21",Tampa,Texas,94143,United States,June 1997,NaN,Interventional,[ 1.26221206e-03 -2.09183618e-02 -5.64291142e-...
1,NCT00000378,Antidepressant Treatment of Melancholia in Lat...,Antidepressant Treatment of Melancholia in Lat...,NaN,The purpose of this study is to compare the sa...,To compare the efficacy and safety of a select...,Inclusion Criteria:\n\n-\n\nPatients must have...,All,60 Years,95 Years,1051 Riverside Drive,New York,New York,10032,United States,NaN,NaN,Interventional,[-4.05032970e-02 -1.93984844e-02 -6.59292657e-...
2,NCT00000620,Action to Control Cardiovascular Risk in Diabe...,Action to Control Cardiovascular Risk in Diabe...,NaN,The purpose of this study is to prevent major ...,"BACKGROUND:\n\nCurrently, about 17 million Ame...",Inclusion Criteria:\n\nDiagnosed with type 2 d...,All,40 Years,79 Years,Wake Forest University,Winston-Salem,Washington,98195,United States,All participants had established type 2 diabet...,Eligible participants provided evidence of abi...,Interventional,[-2.31428873e-02 1.43336959e-03 2.46392819e-...
3,NCT00001151,"Studies With 1,25-Dihydroxycholecalciferol","Studies With 1,25-Dihydroxycholecalciferol",NaN,Vitamin D in the diet undergoes changes in the...,"Patients with extreme resistance to 1,25-dihyd...",INCLUSION CRITERIA:\n\nPatients with hereditar...,All,NaN,NaN,"National Institutes of Health Clinical Center,...",Bethesda,Maryland,20892,United States,NaN,NaN,Interventional,[-4.01057079e-02 6.19083084e-03 1.18229026e-...
4,NCT00001213,Cysteamine Eye Drops to Treat Corneal Crystals...,Trial of Topical Cysteamine in the Treatment o...,Although 328 participants were initially enrol...,Cystinosis is an inherited disease that result...,"Protocol 86-EI-0062 began as a randomized, dou...",INCLUSION CRITERIA:\n\nPatients must have a do...,All,2 Years,NaN,"National Institutes of Health Clinical Center,...",Bethesda,Maryland,20892,United States,NaN,NaN,Interventional,[-8.30913661e-04 -2.10672207e-02 -1.06763479e-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34978,NCT05490771,Testing Copanlisib as a Potential Targeted Tre...,Phase II Study of Copanlisib in Patients With ...,NaN,This phase II MATCH treatment trial identifies...,PRIMARY OBJECTIVE:\n\nI. To evaluate the propo...,Inclusion Criteria:\n\nPatients must have met ...,All,18 Years,NaN,ECOG-ACRIN Cancer Research Group,Philadelphia,Pennsylvania,19103,United States,"Subprotocol Z1F was activated on June 20, 2018...",The PIK3CA mutations status was determined by ...,Interventional,[-1.56701133e-02 -7.25318911e-03 1.66935958e-...
34979,NCT05502081,Clinical Study to Compare Efficacy and Safety ...,Clinical Study to Evaluate the Possible Effica...,0. Uninfected\n\nAmbulatory mild disease\n\nAs...,Introduction:\n\nCorona Virus induced disease ...,I. INTRODUCTION\n\n1.1. COVID-19 overview and ...,Inclusion Criteria:\n\nage more than 12 years ...,All,12 Years,NaN,El-gomhoria St,Mansoura,El-dkhalia,050,Egypt,from 1/11/2021 to 29/5/2022 at isolation hospi...,assignment is applied after admission of parti...,Interventional,[-0.01704628 -0.02524983 0.01437382 0.009319...
34980,NCT05594173,Chewing and Oral Processing of Solid Food,Chewing and Oral Processing of Solid Food in H...,NaN,Food texture modification is commonly used as ...,Aim: To explore chewing and oral processing be...,Inclusion Criteria:\n\nHealthy adults under ag...,All,18 Years,60 Years,Toronto Re

In [50]:
# from ast import literal_eval
# embeddings = df["embeddings"].apply(literal_eval)
embeddings = df["embeddings"].apply(lambda x: np.fromstring(re.sub(r'[\n\[\]]', '', x), sep=' ').astype(np.float32))
embeddings = np.stack(embeddings)
print(embeddings.shape)
print(embeddings[0].dtype)

(34983, 768)
float32


In [51]:
index_column = "nct_id"
embed_column = "embeddings"
vectors = []
for index, row in tqdm(df.iterrows()):
    vectors.append(
        # [row[index_column], row[embed_column], row[1:-1]
        [row[index_column], embeddings[index], row[1:-1]]
    )


0it [00:00, ?it/s]

In [53]:
import vecs

# DB_CONNECTION = "postgresql://<user>:<password>@<host>:<port>/<db_name>"
DB_CONNECTION = f"postgresql://{username}:{password}@{hostname}:{int(port)}/{database}"
# create vector store client
vx = vecs.create_client(DB_CONNECTION)

# create a collection of vectors with 3 dimensions
docs = vx.create_collection(name="ctrial", dimension=embeddings.shape[1])


In [54]:
# index the collection for fast search performance
docs.create_index()


In [61]:
docs.table

Table('ctrial', MetaData(), Column('id', String(), table=<ctrial>, primary_key=True, nullable=False), Column('vec', Vector(dim=768), table=<ctrial>, nullable=False), Column('metadata', JSONB(astext_type=Text()), table=<ctrial>, nullable=False, server_default=DefaultClause(<sqlalchemy.sql.elements.TextClause object at 0x7f2f6536c040>, for_update=False)), schema='vecs')

In [62]:
# query the collection filtering metadata for "year" = 2012
docs.query(
    query_vector=list(embeddings[:,0]),
    limit=2,                         # number of records to return
    # filters={"year": {"$eq": 2012}},  # metadata filters
)


StatementError: (builtins.ValueError) expected 768 dimensions, not 34983
[SQL: SELECT vecs.ctrial.id 
FROM vecs.ctrial 
WHERE (vecs.ctrial.metadata -> %(metadata_1)s) = CAST(%(param_1)s AS JSONB) ORDER BY vecs.ctrial.vec <=> %(vec_1)s 
 LIMIT %(param_2)s]
[parameters: [{}]]